In [1]:
import pandas as pd
import os
import gensim
from tqdm import tqdm
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re

In [2]:
train_data1 =pd.read_csv('Datasets/label_train.csv')
unlabel1 = pd.read_csv('Datasets/unlabel1_final.csv')
unlabel2 = pd.read_csv('Datasets/unlabel2_final.csv')
unlabel3 = pd.read_csv('Datasets/unlabel3_final.csv')
unlabel4 = pd.read_csv('Datasets/unlabel4_final.csv')
unlabel5 = pd.read_csv('Datasets/unlabel5_final.csv')
unlabel6 = pd.read_csv('Datasets/unlabel6_final.csv')
test_file = pd.read_csv('Datasets/test_file.csv')


In [3]:
# # train_data = train_data.dropna()
# unlabel1 = unlabel1.dropna()
train_data1 = train_data1.drop(columns ='Unnamed: 0',axis = 1)
test_file = test_file.drop(columns ='Unnamed: 0',axis = 1)
unlabel1 = unlabel1.drop(columns ='Unnamed: 0',axis = 1)
unlabel2 = unlabel2.drop(columns ='Unnamed: 0',axis = 1)
unlabel3 = unlabel3.drop(columns ='Unnamed: 0',axis = 1)
unlabel4 = unlabel4.drop(columns ='Unnamed: 0',axis = 1)
unlabel5 = unlabel5.drop(columns ='Unnamed: 0',axis = 1)
unlabel6 = unlabel6.drop(columns ='Unnamed: 0',axis = 1)

In [4]:
# Splitting the training data into test and train
train_data, test1 = train_test_split(train_data1, test_size = 0.20)

In [5]:
# Splitting test and train and predictors and labels

X_train = train_data['text']
Y_train = train_data['label']

In [6]:
X_test = test1['text']
Y_test = test1['label']

In [7]:
X1_test = unlabel1['text']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vectoriser = TfidfVectorizer(sublinear_tf = True,
                               analyzer = 'word',
                               max_df = 0.95,
                               min_df = 10,
                               ngram_range = (1,2),use_idf = True)

x_train = tf_vectoriser.fit_transform(X_train)
x_test = tf_vectoriser.transform(X_test)
x1_test = tf_vectoriser.transform(X1_test)

print("Vocab size: {}".format(len(tf_vectoriser.vocabulary_)))
print("x_train: {}".format(repr(x_train)))
print("x_test: {}".format(repr(x1_test)))

feature_names = tf_vectoriser.get_feature_names()
print("Number of features: {}".format(len(feature_names)))

Vocab size: 65406
x_train: <40000x65406 sparse matrix of type '<class 'numpy.float64'>'
	with 5679409 stored elements in Compressed Sparse Row format>
x_test: <100000x65406 sparse matrix of type '<class 'numpy.float64'>'
	with 14084484 stored elements in Compressed Sparse Row format>
Number of features: 65406


In [9]:
# Adding emoticons for training

sad = np.array(train_data['sad'])
print("Sad:",sad.shape)
smile = np.array(train_data['smile'])
print("Smile:",smile.shape)
laugh = np.array(train_data['laugh'])
print("Laugh",laugh.shape)
very_happy = np.array(train_data['very_happy'])
print("Very happy:",very_happy.shape)
crying = np.array(train_data['crying'])
print("crying:",crying.shape)
tongueOut = np.array(train_data['tongueOut'])
print("tongueOut:",tongueOut.shape)
wink = np.array(train_data['wink'])
print("wink:",wink.shape)
kiss = np.array(train_data['kiss'])
print("kiss:",kiss.shape)
surprise = np.array(train_data['surprise'])
print("surprise:",surprise.shape)
disgust = np.array(train_data['disgust'])
print("disgust:",disgust.shape)
exclamation = np.array(train_data['exclamation'])
print("exclamation:",exclamation.shape)
capital_Ratio = np.array(train_data['capital_Ratio'])
print("capital_Ratio:",capital_Ratio.shape)

Sad: (40000,)
Smile: (40000,)
Laugh (40000,)
Very happy: (40000,)
crying: (40000,)
tongueOut: (40000,)
wink: (40000,)
kiss: (40000,)
surprise: (40000,)
disgust: (40000,)
exclamation: (40000,)
capital_Ratio: (40000,)


In [10]:
# Adding emoticons for TEST

sad1 = np.array(test1['sad'])
print("Sad:",sad1.shape)
smile1 = np.array(test1['smile'])
print("Smile:",smile1.shape)
laugh1 = np.array(test1['laugh'])
print("Laugh",laugh1.shape)
very_happy1 = np.array(test1['very_happy'])
print("Very happy:",very_happy1.shape)
crying1 = np.array(test1['crying'])
print("crying:",crying1.shape)
tongueOut1 = np.array(test1['tongueOut'])
print("tongueOut:",tongueOut1.shape)
wink1 = np.array(test1['wink'])
print("wink:",wink1.shape)
kiss1 = np.array(test1['kiss'])
print("kiss:",kiss1.shape)
surprise1 = np.array(test1['surprise'])
print("surprise:",surprise1.shape)
disgust1 = np.array(test1['disgust'])
print("disgust:",disgust1.shape)
exclamation1 = np.array(test1['exclamation'])
print("exclamation:",exclamation1.shape)
capital_Ratio1 = np.array(test1['capital_Ratio'])
print("capital_Ratio:",capital_Ratio1.shape)

Sad: (10000,)
Smile: (10000,)
Laugh (10000,)
Very happy: (10000,)
crying: (10000,)
tongueOut: (10000,)
wink: (10000,)
kiss: (10000,)
surprise: (10000,)
disgust: (10000,)
exclamation: (10000,)
capital_Ratio: (10000,)


In [11]:
# Adding emoticons for X1_test (unlabled data)

sad2 = np.array(unlabel1['sad'])
print("Sad:",sad2.shape)
smile2 = np.array(unlabel1['smile'])
print("Smile:",smile2.shape)
laugh2 = np.array(unlabel1['laugh'])
print("Laugh",laugh2.shape)
very_happy2 = np.array(unlabel1['very_happy'])
print("Very happy:",very_happy2.shape)
crying2 = np.array(unlabel1['crying'])
print("crying:",crying2.shape)
tongueOut2 = np.array(unlabel1['tongueOut'])
print("tongueOut:",tongueOut2.shape)
wink2 = np.array(unlabel1['wink'])
print("wink:",wink2.shape)
kiss2 = np.array(unlabel1['kiss'])
print("kiss:",kiss2.shape)
surprise2 = np.array(unlabel1['surprise'])
print("surprise:",surprise2.shape)
disgust2 = np.array(unlabel1['disgust'])
print("disgust:",disgust2.shape)
exclamation2 = np.array(unlabel1['exclamation'])
print("exclamation:",exclamation2.shape)
capital_Ratio2 = np.array(unlabel1['capital_Ratio'])
print("capital_Ratio:",capital_Ratio2.shape)

Sad: (100000,)
Smile: (100000,)
Laugh (100000,)
Very happy: (100000,)
crying: (100000,)
tongueOut: (100000,)
wink: (100000,)
kiss: (100000,)
surprise: (100000,)
disgust: (100000,)
exclamation: (100000,)
capital_Ratio: (100000,)


### Adding features to sparse matrix

In [12]:
from scipy.sparse import hstack

# Adding features for training file

X_train_dtm = hstack((x_train,sad[:,None]))
X_train_dtm = hstack((X_train_dtm,smile[:,None]))
X_train_dtm = hstack((X_train_dtm,laugh[:,None]))
X_train_dtm = hstack((X_train_dtm,very_happy[:,None]))
X_train_dtm = hstack((X_train_dtm,crying[:,None]))
X_train_dtm = hstack((X_train_dtm,tongueOut[:,None]))
X_train_dtm = hstack((X_train_dtm,wink[:,None]))
X_train_dtm = hstack((X_train_dtm,kiss[:,None]))
X_train_dtm = hstack((X_train_dtm,surprise[:,None]))
X_train_dtm = hstack((X_train_dtm,disgust[:,None]))
X_train_dtm = hstack((X_train_dtm,exclamation[:,None]))
X_train_dtm = hstack((X_train_dtm,capital_Ratio[:,None]))

In [13]:
# Adding features for testing file

X_test_dtm = hstack((x_test,sad1[:,None]))
X_test_dtm = hstack((X_test_dtm,smile1[:,None]))
X_test_dtm = hstack((X_test_dtm,laugh1[:,None]))
X_test_dtm = hstack((X_test_dtm,very_happy1[:,None]))
X_test_dtm = hstack((X_test_dtm,crying1[:,None]))
X_test_dtm = hstack((X_test_dtm,tongueOut1[:,None]))
X_test_dtm = hstack((X_test_dtm,wink1[:,None]))
X_test_dtm = hstack((X_test_dtm,kiss1[:,None]))
X_test_dtm = hstack((X_test_dtm,surprise1[:,None]))
X_test_dtm = hstack((X_test_dtm,disgust1[:,None]))
X_test_dtm = hstack((X_test_dtm,exclamation1[:,None]))
X_test_dtm = hstack((X_test_dtm,capital_Ratio1[:,None]))

In [14]:
# Adding features for X1_test file (unlabled data)

X1_test_dtm = hstack((x1_test,sad2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,smile2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,laugh2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,very_happy2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,crying2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,tongueOut2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,wink2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,kiss2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,surprise2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,disgust2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,exclamation2[:,None]))
X1_test_dtm = hstack((X1_test_dtm,capital_Ratio2[:,None]))

### Logistic regression

In [15]:
clf = LogisticRegression()
clf.fit(X_train_dtm, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
y_pred = clf.predict(X_test_dtm)

In [17]:
from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(Y_test, y_pred))

Accuracy :  0.6181


### Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train_dtm,Y_train)

C:\Users\Rachana.LAPTOP-2JO12NAC\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
y_pred_rf = rf.predict(X_test_dtm)

In [20]:
from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(Y_test, y_pred_rf))

Accuracy :  0.4249


### XGBoost

In [21]:
import xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train_dtm,Y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [22]:
predxgb = xgb.predict(X_test_dtm)

C:\Users\Rachana.LAPTOP-2JO12NAC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
# from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(Y_test, predxgb))

Accuracy :  0.5267


### Multi-layer perceptron

In [22]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train_dtm,Y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [23]:
pred_mlp = mlp.predict(X_test_dtm)

In [24]:
# from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(Y_test, pred_mlp))

Accuracy :  0.5336


### Gradient Booster

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

gbi = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,max_features=0.5,random_state=999999)
gbi.fit(X_train_dtm,Y_train)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=0.5, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=999999, subsample=1.0,
              verbose=0, warm_start=False)

In [26]:
predgbi = gbi.predict(X_test_dtm)

In [27]:
# from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(Y_test, predgbi))

Accuracy :  0.5543


### Logistic regression

In [28]:
clf = LogisticRegression()
clf.fit(X_train_dtm, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
y_pred = clf.predict(X_test_dtm)

In [30]:
from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(Y_test, y_pred))

Accuracy :  0.6025
